In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Preparing match DF

In [3]:
path = 'data/database.sqlite'

In [4]:
connection = sqlite3.connect(path)

In [5]:
query = '''
  SELECT * 
  FROM Match
'''
match_df = pd.read_sql_query(query, connection)
match_df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [6]:
format_date_str = "%Y-%m-%d %H:%M:%S"
match_df['date'] = match_df['date'].apply(lambda x: datetime.strptime(x, format_date_str))


In [7]:
identification_cols = ['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'goal', 'shoton', 'shotoff', 'possession']
betting_cols = list(match_df.columns[-30:]) 
cols = identification_cols + betting_cols
match_df = match_df[cols]

In [8]:
for res in ['H', 'D', 'A']:
    match_df[f'mean_bet_{res}'] = match_df[[col for col in betting_cols if col[-1] == res]].mean(axis=1)

C:\Users\EB\AppData\Local\Temp/ipykernel_21300/3007036654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df[f'mean_bet_{res}'] = match_df[[col for col in betting_cols if col[-1] == res]].mean(axis=1)


In [9]:
match_df = match_df[identification_cols + [f'mean_bet_{res}' for res in ['H', 'D', 'A']]]

In [10]:
print(list(match_df.columns))

['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'goal', 'shoton', 'shotoff', 'possession', 'mean_bet_H', 'mean_bet_D', 'mean_bet_A']


A df with all the wins, we can use this as part of x to describe how good a team is in a given season


In [11]:
wins = pd.read_sql_query("""
  SELECT season, home_team_api_id, SUM(home_win + away_win) as wins, SUM(draw) as draws, league_id
  FROM
    (
    SELECT MATCH.id, MATCH.home_team_api_id, Match.season, t1.team_long_name as home_team, t2.team_long_name as away_team,
      (CASE WHEN home_team_goal > away_team_goal THEN 1 ELSE 0 END) AS home_win,
      (CASE WHEN home_team_goal = away_team_goal THEN 1 ELSE 0 END) AS draw,
      (CASE WHEN home_team_goal < away_team_goal THEN 1 ELSE 0 END) AS away_win, league_id
    FROM MATCH 
    JOIN TEAM as t1 ON t1.team_api_id = MATCH.home_team_api_id 
    JOIN TEAM as t2 ON t2.team_api_id = MATCH.away_team_api_id)
    GROUP BY home_team, season
    ORDER BY season DESC, wins DESC
  """, connection)

In [12]:
wins['league_points'] = 3*wins['wins'] + wins['draws'] 
wins

,season,home_team_api_id,wins,draws,league_id,league_points
0,2015/2016,8649,19,0,19694,57
1,2015/2016,10249,19,0,4769,57
2,2015/2016,8302,18,1,21518,55
3,2015/2016,8633,18,1,21518,55
4,2015/2016,8634,18,1,21518,55
...,...,...,...,...,...,...
1473,2008/2009,9888,9,10,10257,37
1474,2008/2009,9912,9,8,7809,35
1475,2008/2009,2186,9,6,15722,33
1476,2008/2009,2182,8,7,15722,31


In [13]:
match_df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,goal,shoton,shotoff,possession,mean_bet_H,mean_bet_D,mean_bet_A
0,1,1,1,2008/2009,1,2008-08-17,492473,9987,9993,1,1,None,None,None,None,1.765556,3.322222,4.164444
1,2,1,1,2008/2009,1,2008-08-16,492474,10000,9994,0,0,None,None,None,None,1.898889,3.250000,3.616667
2,3,1,1,2008/2009,1,2008-08-16,492475,9984,8635,0,3,None,None,None,None,2.462222,3.233333,2.544444
3,4,1,1,2008/2009,1,2008-08-17,492476,9991,9998,5,0,None,None,None,None,1.438889,3.805556,6.533333
4,5,1,1,2008/2009,1,2008-08-16,492477,7947,9985,1,3,None,None,None,None,4.494444,3.422222,1.674444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22,1992091,10190,10191,1,0,None,None,None,None,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23,1992092,9824,10199,1,2,None,None,None,None,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23,1992093,9956,10179,2,0,None,None,None,None,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22,1992094,7896,10243,0,0,None,None,None,None,NaN,NaN,NaN


In [14]:
win_cols = ['wins', 'draws', 'league_points']
wins_home = wins.copy().rename({col: f"{col}_home" for col in win_cols}, axis=1)
wins_away = wins.copy().rename({col: f"{col}_away" for col in win_cols}, axis=1)
wins_away = wins_away.rename({'home_team_api_id': 'away_team_api_id'}, axis=1)
match_df = match_df.merge(wins_home, on=['league_id', 'season', 'home_team_api_id'])
match_df = match_df.merge(wins_away, on=['league_id', 'season', 'away_team_api_id'])

match_df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,goal,shoton,shotoff,possession,mean_bet_H,mean_bet_D,mean_bet_A,wins_home,draws_home,league_points_home,wins_away,draws_away,league_points_away
0,1,1,1,2008/2009,1,2008-08-17,492473,9987,9993,1,1,None,None,None,None,1.765556,3.322222,4.164444,12,5,41,14,3,45
1,30,1,1,2008/2009,12,2008-11-15,492584,10000,9993,2,2,None,None,None,None,2.022222,3.227778,3.322222,11,6,39,14,3,45
2,264,1,1,2008/2009,5,2008-09-20,492519,9984,9993,1,0,None,None,None,None,2.012222,3.216667,3.322222,15,2,47,14,3,45
3,144,1,1,2008/2009,23,2009-02-21,493015,9991,9993,1,3,None,None,None,None,1.701111,3.333333,4.453333,14,3,45,14,3,45
4,99,1,1,2008/2009,19,2009-01-24,492819,7947,9993,0,1,None,None,None,None,2.797778,3.216667,2.274444,13,4,43,14,3,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25884,25976,24558,24558,2015/2016,9,2015-09-23,1992092,9824,10199,1,2,None,None,None,None,NaN,NaN,NaN,7,11,32,13,5,44
25885,25855,24558,24558,2015/2016,2,2015-07-25,1992056,10192,10199,1,1,None,None,None,None,NaN,NaN,NaN,14,4,46,13,5,44
25886,25878,24558,24558,2015/2016,23,2016-03-06,1992164,10192,10199,5,2,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<possession />,NaN,NaN,NaN,14,4,46,13,5,44
25887,25922,24558,24558,2015/2016,31,2016-05-01,1992203,7896,10199,1,1,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<possession />,NaN,NaN,NaN,14,4,46,13,5,44


In [18]:
df['goal_diff'] = df['home_team_goal'] - df['away_team_goal']
df = df[['match_api_id', 'season', 'date', 'home_team_api_id', 'away_team_api_id', 'goal_diff', 'league_points_home', 
         'league_points_away', 'mean_bet_H', 'mean_bet_D', 'mean_bet_A']]

In [19]:
league_constant_cols = ['season']
groups = df.groupby('home_team_api_id')
joined_games = []
for group_id, group_df in groups:
    for season, season_df in group_df.groupby('season'):
        season_df = season_df.sort_values('date')
        next_game = season_df.iloc[1:].copy().set_index(season_df.index[:-1])
        display(next_game)
        prev_game = season_df.iloc[:-1].copy()
        display(prev_game)
        prev_game[[f'next_game_{col}' for col in next_game.columns]] = next_game
        prev_game = prev_game.drop([f'next_game_{col}' for col in league_constant_cols], axis=1)
        joined_games.append(prev_game)
game_tuples_df = pd.concat(joined_games)
game_tuples_df

,match_api_id,season,date,home_team_api_id,away_team_api_id,goal_diff,league_points_home,league_points_away,mean_bet_H,mean_bet_D,mean_bet_A
750,658601,2009/2010,2009-08-23,8191,8668,1,47,45,3.194444,3.244444,2.227778
427,658631,2009/2010,2009-09-19,8191,8472,2,47,43,2.767778,3.227778,2.494444
713,658684,2009/2010,2009-10-03,8191,8658,1,47,39,2.227778,3.238889,3.133333
390,658699,2009/2010,2009-10-24,8191,8528,-2,47,43,2.397778,3.250000,2.895556
485,658713,2009/2010,2009-10-31,8191,8667,2,47,45,1.912222,3.333333,3.992222
523,658792,2009/2010,2009-11-21,8191,10252,0,47,41,3.244444,3.288889,2.205556
446,658826,2009/2010,2009-12-12,8191,9879,0,47,51,2.583333,3.238889,2.685556
580,658834,2009/2010,2009-12-16,8191,9825,0,47,53,6.777778,4.064444,1.487778
675,658923,2009/2010,2009-12-26,8191,8559,0,47,45,2.313333,3.261111,3.007778
731,658973,2009/2010,2010-01-30,8191,8455,-1,47,55,11.111111,4.950000,1.287778


,match_api_id,season,date,home_team_api_id,away_team_api_id,goal_diff,league_points_home,league_points_away,mean_bet_H,mean_bet_D,mean_bet_A
750,658589,2009/2010,2009-08-19,8191,10260,1,47,55,9.944444,4.722222,1.317778
427,658601,2009/2010,2009-08-23,8191,8668,1,47,45,3.194444,3.244444,2.227778
713,658631,2009/2010,2009-09-19,8191,8472,2,47,43,2.767778,3.227778,2.494444
390,658684,2009/2010,2009-10-03,8191,8658,1,47,39,2.227778,3.238889,3.133333
485,658699,2009/2010,2009-10-24,8191,8528,-2,47,43,2.397778,3.250000,2.895556
523,658713,2009/2010,2009-10-31,8191,8667,2,47,45,1.912222,3.333333,3.992222
446,658792,2009/2010,2009-11-21,8191,10252,0,47,41,3.244444,3.288889,2.205556
580,658826,2009/2010,2009-12-12,8191,9879,0,47,51,2.583333,3.238889,2.685556
675,658834,2009/2010,2009-12-16,8191,9825,0,47,53,6.777778,4.064444,1.487778
731,658923,2009/2010,2009-12-26,8191,8559,0,47,45,2.313333,3.261111,3.007778


ZeroDivisionError: division by zero

In [ ]:
game_tuples_df

In [31]:
pd.read_pickle('data/consecutive_games_df.pkl')

,row_team,season,league_id,row_team_wins,row_team_draws,row_team_league_points,prev_match_api_id,prev_date,prev_home,prev_goal_diff,prev_mean_bet_row_team,prev_mean_bet_rival,prev_mean_bet_draw,prev_rival_team_api_id,prev_rival_wins,prev_rival_draws,prev_rival_league_points,next_match_api_id,next_date,next_home,next_goal_diff,next_mean_bet_row_team,next_mean_bet_rival,next_mean_bet_draw,next_rival_team_api_id,next_rival_wins,next_rival_draws,next_rival_league_points
0,8559,2008/2009,1729,14,5,47,489049,2008-08-16,True,2,1.793333,4.340000,3.300000,10194,14,5,47,489054,2008-08-23,False,-1,4.577778,1.786667,3.400000,10261,12,7,43
1,8528,2008/2009,1729,14,5,47,489045,2008-08-16,False,-1,3.938889,1.874444,3.283333,8654,17,2,53,489061,2008-08-24,True,-1,7.944444,1.405556,4.264444,8455,13,6,45
1,8654,2008/2009,1729,17,2,53,489045,2008-08-16,True,1,1.874444,3.938889,3.283333,8528,14,5,47,489055,2008-08-24,False,-3,3.194444,2.222222,3.238889,8456,19,0,57
2,8655,2008/2009,1729,12,7,43,489047,2008-08-16,False,1,3.716667,1.961111,3.233333,8668,13,6,45,489056,2008-08-23,True,0,1.583333,6.033333,3.638889,8667,14,5,47
4,8659,2008/2009,1729,16,3,51,489043,2008-08-16,False,-1,12.222222,1.208889,5.522222,9825,14,5,47,489058,2008-08-23,True,-1,2.775556,2.485556,3.227778,8668,13,6,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3028,8472,2015/2016,1729,13,6,45,1989008,2016-05-11,True,3,1.707143,5.000000,3.828571,8668,14,5,47,1987605,2016-05-15,False,0,3.582857,2.054286,3.628571,9817,13,6,45
3027,8455,2015/2016,1729,10,9,39,1989004,2016-05-11,False,0,3.744286,1.961429,3.644286,8650,11,8,41,1987598,2016-05-15,True,0,2.268571,3.081429,3.601429,8197,13,6,45
3027,8650,2015/2016,1729,11,8,41,1989004,2016-05-11,True,0,1.961429,3.744286,3.644286,8455,10,9,39,1987606,2016-05-15,False,0,2.902857,2.407143,3.500000,8659,14,5,47
3029,9850,2015/2016,1729,14,5,47,1989056,2016-05-11,True,2,1.808571,4.537143,3.652857,9817,13,6,45,1987599,2016-05-15,False,-3,4.387143,1.785714,3.918571,8668,14,5,47
